# **[Demo] Create a `dlt` pipeline in 4 steps** 🚀

---

**TL;DR: create and execute a pipeline that loads data from the [PokeAPI](https://pokeapi.co/) into a [DuckDB](https://duckdb.org/) database using the `dlt` library**

## 1 **Install `dlt`**

In [4]:
%%capture
!pip install "dlt[duckdb]" # Install dlt with all the necessary DuckDB dependencies

In [1]:
!dlt --version

dlt 0.3.23


## 2 **Import `dlt` and initialize the pipeline**

In [2]:
import dlt

pipeline = dlt.pipeline(pipeline_name="csd_pipeline",
                        destination="duckdb",
                        dataset_name="csdb_releases")

## 3 **Get data from the source**

We load the list of pokemons data using url.

In [18]:
from dlt.sources.helpers import requests
import xml.etree.ElementTree as ET

POKEMON_URL = "https://csdb.dk/webservice/?type=release&id=1&depth=1"

response = requests.get(POKEMON_URL)
if response.status_code == 200:
    root = ET.fromstring(response.content)

    # Extract and transform 'results' data into list of dictionaries
    results = root.findall(".//results")  # Use correct path based on XML
    data = []
    for result in results:
        data.append({
            "name": result.find("name").text if result.find("name") is not None else None,
            "type": result.find("type").text if result.find("type") is not None else None,
            # Add all relevant fields based on the XML tags
        })

    print("Formatted Data for Pipeline:", data)
else:
    print(f"Failed to fetch data. HTTP Status Code: {response.status_code}")
    data = []  # Ensure data is at least an empty list to avoid further errors


Formatted Data for Pipeline: []


## 4 **Run the pipeline**

In [19]:
# Check data before running the pipeline
print("Data Preview:", data[:5])  # Show a sample of the first 5 records

# Run the pipeline with normalized data
load_info = pipeline.run(data, table_name='pokemon')
print(load_info)

Data Preview: []
Pipeline csd_pipeline completed in 0.69 seconds
1 load package(s) were loaded to destination duckdb and into dataset csdb_releases
The duckdb destination used duckdb:////Users/rolf/Documents/src/GitHub/dlt-test/csd_pipeline.duckdb location to store data
Load package 1734797883.271083 is LOADED and contains no failed jobs


In [20]:
# Normalize and load the data onto the locally created duckdb database 'pokemon_pipeline.duckdb'
load_info = pipeline.run(data, table_name='pokemon')
print(load_info)

Pipeline csd_pipeline completed in 0.03 seconds
0 load package(s) were loaded to destination duckdb and into dataset None
The duckdb destination used duckdb:////Users/rolf/Documents/src/GitHub/dlt-test/csd_pipeline.duckdb location to store data


## 🎉 **Finish!** 🎉

## **Bonus: Query the loaded data** 🦆

To access the loaded data, connect to the DuckDB database using the Python DuckDB connector

In [21]:
import duckdb
import polars as pl  # Import Polars for DataFrame manipulations

# Connect to the DuckDB database
conn = duckdb.connect(f"{pipeline.pipeline_name}.duckdb")

# Query and convert result to a Polars DataFrame
describe_result = pl.DataFrame(
    conn.sql("DESCRIBE").fetchall(),
    schema=conn.sql("DESCRIBE").columns,
    orient="row"  # Specify row orientation explicitly
)

# Display the Polars DataFrame
print(describe_result)


shape: (3, 6)
┌──────────────┬───────────────┬────────────────────┬───────────────────┬──────────────┬───────────┐
│ database     ┆ schema        ┆ name               ┆ column_names      ┆ column_types ┆ temporary │
│ ---          ┆ ---           ┆ ---                ┆ ---               ┆ ---          ┆ ---       │
│ str          ┆ str           ┆ str                ┆ list[str]         ┆ list[str]    ┆ bool      │
╞══════════════╪═══════════════╪════════════════════╪═══════════════════╪══════════════╪═══════════╡
│ csd_pipeline ┆ csdb_releases ┆ _dlt_loads         ┆ ["load_id",       ┆ ["VARCHAR",  ┆ false     │
│              ┆               ┆                    ┆ "schema_name", …  ┆ "VARCHAR", … ┆           │
│              ┆               ┆                    ┆ "…                ┆ "VARC…       ┆           │
│ csd_pipeline ┆ csdb_releases ┆ _dlt_pipeline_stat ┆ ["version",       ┆ ["BIGINT",   ┆ false     │
│              ┆               ┆ e                  ┆ "engine_version", ┆ "BI

In [22]:
import duckdb
import polars as pl

# Example in-memory data
pokemon_data = [(1, "bulbasaur"), (2, "ivysaur"), (3, "venusaur")]

# Connect to DuckDB (in-memory)
conn = duckdb.connect()

# Step 1: Create the table schema
conn.execute("CREATE TABLE pokemon_table (id INT, name VARCHAR)")

# Step 2: Insert data using prepared statements
for record in pokemon_data:
    conn.execute("INSERT INTO pokemon_table VALUES (?, ?)", record)

# Step 3: Query data directly and load into a Polars DataFrame
result = pl.DataFrame(
    conn.execute("SELECT * FROM pokemon_table").fetchall(),
    schema=["id", "name"],
    orient="row",  # Explicitly specify row orientation
)

print(result)


shape: (3, 2)
┌─────┬───────────┐
│ id  ┆ name      │
│ --- ┆ ---       │
│ i64 ┆ str       │
╞═════╪═══════════╡
│ 1   ┆ bulbasaur │
│ 2   ┆ ivysaur   │
│ 3   ┆ venusaur  │
└─────┴───────────┘


In [24]:
# this lets us query data without adding schema prefix to table names
conn.sql(f"SET search_path = '{pipeline.dataset_name}'")

# Retrieve SQL query results and convert them directly into a Polars DataFrame
stats_table = conn.sql("SELECT * FROM pokemon").pl()  # .pl() assumes Polars compatibility here

# Display the Polars DataFrame
print(stats_table)


shape: (20, 4)
┌────────────┬─────────────────────────────────┬───────────────────┬────────────────┐
│ name       ┆ url                             ┆ _dlt_load_id      ┆ _dlt_id        │
│ ---        ┆ ---                             ┆ ---               ┆ ---            │
│ str        ┆ str                             ┆ str               ┆ str            │
╞════════════╪═════════════════════════════════╪═══════════════════╪════════════════╡
│ bulbasaur  ┆ https://pokeapi.co/api/v2/poke… ┆ 1734793981.588002 ┆ bAvKtRkBARAxwg │
│ ivysaur    ┆ https://pokeapi.co/api/v2/poke… ┆ 1734793981.588002 ┆ CvzTJVbcInbpuA │
│ venusaur   ┆ https://pokeapi.co/api/v2/poke… ┆ 1734793981.588002 ┆ rwJZT1m/2u19/g │
│ charmander ┆ https://pokeapi.co/api/v2/poke… ┆ 1734793981.588002 ┆ ei8/E+h2co+Wkg │
│ charmeleon ┆ https://pokeapi.co/api/v2/poke… ┆ 1734793981.588002 ┆ Bbfe9TBXvs0HAA │
│ …          ┆ …                               ┆ …                 ┆ …              │
│ pidgey     ┆ https://pokeapi.co/api/v

In [25]:
conn.close()


# **Next Steps** 👀

## **[Give `dlt` a ⭐ on GitHub](https://github.com/dlt-hub/dlt)**
## **[Join the `dlt` community on Slack](https://join.slack.com/t/dlthub-community/shared_invite/zt-1slox199h-HAE7EQoXmstkP_bTqal65g)**
## **[Read the `dlt` docs](https://dlthub.com/docs/)**